In [86]:
# Importing the library
import pandas as pd 

In [87]:
# Reading data from CSV files
sale_data = pd.read_csv("C:/Users/black/OneDrive/Desktop/Data_Integration_with_Python/sales_data.csv", header=0)
customer_data = pd.read_csv("C:/Users/black/OneDrive/Desktop/Data_Integration_with_Python/customer_data.csv", header=0)
product_data = pd.read_csv("C:/Users/black/OneDrive/Desktop/Data_Integration_with_Python/product_data.csv", header=0)
shipping_data = pd.read_csv("C:/Users/black/OneDrive/Desktop/Data_Integration_with_Python/shipping_data.csv", header=0)

In [88]:
sale_data.head()

,Order_ID,Date,Product,Amount,Customer_ID
0,Order_1,1/1/2023,Product_A,120,Customer_1
1,Order_2,1/2/2023,Product_B,180,Customer_2
2,Order_3,1/3/2023,Product_A,150,Customer_3
3,Order_4,1/4/2023,Product_C,90,Customer_4
4,Order_5,1/5/2023,Product_B,200,Customer_5


In [89]:
customer_data.head()

,Customer_ID,Customer_Name,Region,Gender,Age,Segment,Loyalty_level
0,Customer_1,John Doe,North,Male,32,Gold,Platinum
1,Customer_2,Jane Smith,South,Female,28,Silver,Gold
2,Customer_3,Bob Johnson,West,Male,40,Platinum,Silver
3,Customer_4,Alice Brown,East,Female,35,Gold,Bronze
4,Customer_5,Charlie Davis,Central,Male,45,Silver,Platinum


In [90]:
shipping_data.head()

,Order_ID,Shipping_Cost,Shipping_Method,Shipping_Region
0,Order_1,10,Express,North
1,Order_2,15,Standard,South
2,Order_3,8,Express,West
3,Order_4,12,Standard,East
4,Order_5,18,Express,North


In [91]:
product_data.head()

,Product,Category,Base_Price,Discount,Supplier,Popularity_Score
0,Product_A,Electronics,150,0.10,Supplier_X,8
1,Product_B,Home & Kitchen,120,0.05,Supplier_Y,6
2,Product_C,Clothering,80,0.20,Supplier_Z,9
3,Product_D,Electronics,200,0.15,Supplier_X,7
4,Product_E,Home & Kitchen,90,0.10,Supplier_Y,5


In [92]:
# performing multi-step data integration process
merged_data = pd.merge(sale_data, customer_data, on='Customer_ID', how='left')

In [93]:
# Extracting year, month, and quarter form the data
merged_data['YearMonth'] = pd.to_datetime(merged_data['Date']).dt.to_period('M')
merged_data['Quarter'] = pd.to_datetime(merged_data['Date']).dt.quarter

In [94]:
# Merge with product data
merged_data = pd.merge(merged_data, product_data, on='Product', how='left')

In [95]:
# Merge with shipping data
merged_data = pd.merge(merged_data, shipping_data, on='Order_ID', how='left')

In [96]:
merged_data.head()

,Order_ID,Date,Product,Amount,Customer_ID,Customer_Name,Region,Gender,Age,Segment,...,YearMonth,Quarter,Category,Base_Price,Discount,Supplier,Popularity_Score,Shipping_Cost,Shipping_Method,Shipping_Region
0,Order_1,1/1/2023,Product_A,120,Customer_1,John Doe,North,Male,32,Gold,...,2023-01,1,Electronics,150,0.10,Supplier_X,8,10,Express,North
1,Order_2,1/2/2023,Product_B,180,Customer_2,Jane Smith,South,Female,28,Silver,...,2023-01,1,Home & Kitchen,120,0.05,Supplier_Y,6,15,Standard,South
2,Order_3,1/3/2023,Product_A,150,Customer_3,Bob Johnson,West,Male,40,Platinum,...,2023-01,1,Electronics,150,0.10,Supplier_X,8,8,Express,West
3,Order_4,1/4/2023,Product_C,90,Customer_4,Alice Brown,East,Female,35,Gold,...,2023-01,1,Clothering,80,0.20,Supplier_Z,9,12,Standard,East
4,Order_5,1/5/2023,Product_B,200,Customer_5,Charlie Davis,Central,Male,45,Silver,...,2023-01,1,Home & Kitchen,120,0.05,Supplier_Y,6,18,Express,North


In [97]:
# Calculate discounted amount based on customer segment, loyalty level, and dynamic pricing
merged_data['Segment_Discount'] = merged_data.apply(
    lambda row: 0.1 if row['Segment'] == 'Gold' else (0.05 if row['Segment'] == 'Silver' else 0), axis=1
)

merged_data['Loyalty_Discount'] = merged_data.apply(
    lambda row: 0.15 if row['Loyalty_level'] == 'Platinum' else (0.1 if row['Loyalty_level'] == 'Gold' else 0), axis=1
)

merged_data['Dynamic_Pricing'] = merged_data.apply(
    lambda row: row['Base_Price'] * (0.1 * row['Popularity_Score']), axis=1
)

merged_data['Discounted_Amount'] = merged_data['Amount'] * (1 - merged_data['Discount'] - merged_data['Segment_Discount'] - merged_data['Loyalty_Discount']) - merged_data['Dynamic_Pricing']

In [98]:
# Calculating total cost including shipping and discounted amount
merged_data['Total_Cost'] = merged_data['Discounted_Amount'] + merged_data['Shipping_Cost']

In [99]:
print(merged_data.columns)

Index(['Order_ID', 'Date', 'Product', 'Amount', 'Customer_ID', 'Customer_Name',
       'Region', 'Gender', 'Age', 'Segment', 'Loyalty_level', 'YearMonth',
       'Quarter', 'Category ', 'Base_Price', 'Discount', 'Supplier',
       'Popularity_Score', 'Shipping_Cost', 'Shipping_Method',
       'Shipping_Region', 'Segment_Discount', 'Loyalty_Discount',
       'Dynamic_Pricing', 'Discounted_Amount', 'Total_Cost'],
      dtype='object')


In [100]:
print('Customer_Name' in merged_data.columns)

True


In [101]:
# Displacing the final integrated data with customer names in the second column
output_data = merged_data[['Order_ID', 'Customer_Name', 'Date', 'Product', 'Amount', 'Region', 'Gender', 'Age', 'Segment', 'Loyalty_level', 'Category ', 'Base_Price', 'Discount', 'Supplier', 'Popularity_Score','Shipping_Cost', 'Shipping_Method', 'Shipping_Region', 'Discounted_Amount', 'Total_Cost']]
print(output_data)

    Order_ID  Customer_Name       Date    Product  Amount   Region  Gender  \
0    Order_1       John Doe   1/1/2023  Product_A     120    North    Male   
1    Order_2     Jane Smith   1/2/2023  Product_B     180    South  Female   
2    Order_3    Bob Johnson   1/3/2023  Product_A     150     West    Male   
3    Order_4    Alice Brown   1/4/2023  Product_C      90     East  Female   
4    Order_5  Charlie Davis   1/5/2023  Product_B     200  Central    Male   
5    Order_6     Emma White   1/6/2023  Product_A     110    North  Female   
6    Order_7      David Lee   1/7/2023  Product_C     130    South    Male   
7    Order_8   Sarah Taylor   1/8/2023  Product_A     160     West  Female   
8    Order_9  Micheal Clark   1/9/2023  Product_B     220     East    Male   
9   Order_10    Olivia Hall  1/10/2023  Product_C     120  Central  Female   
10  Order_11       Mary Toe  1/11/2023  Product_A     100    North  Female   

    Age   Segment Loyalty_level       Category   Base_Price  Di